In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import transformers
from transformers import pipeline
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import numpy as np
import nltk.data
from requests_html import HTMLSession

In [53]:
def parse_href(url, element):
    session = HTMLSession()
    r = session.get(url)
    elements = r.html.find(element)
    links = [element.attrs['href'] for element in elements if 'href' in element.attrs]
    return links


In [ ]:
hyperlinks = parse_href(url, 'a')

# Print the string versions of the hyperlinks
for hyperlink in hyperlinks:
    print(hyperlink)


In [55]:
def parse_elements(url,element):
    session = HTMLSession()
    r = session.get(url)
    elements = r.html.find(element)
    return elements

In [56]:
paragraphs = parse_elements(url, 'p')
links = parse_elements(url, 'a')

In [57]:
def topic_detection(sentence):
    DataScience_words = ('data science', 'regression', 'predict', 'predictive', 'classify')
    AI_words = ('AI', 'machine learning', 'ml', 'ML', 'NLP', 'LLM','deep learning','neural network','generative', 'GPT-3')
    DataWords = ('analysis', 'algorithms','data','data structure','data-structure','big data')
    DataScience = any(sentence.count(i) > 0 for i in DataScience_words)
    AI = any(sentence.count(i) > 0 for i in AI_words)
    data = any(sentence.count(i) > 0 for i in DataWords)
    topics = []
    if DataScience == True:
        topics.append("Data Science")
    if AI == True:
        topics.append("AI")
    if data == True:
        topics.append("data")
    return topics

In [58]:
p_dictionary = {}
p_list = []
topic_list = []
for p in range(0, len(paragraphs)):
    if len(paragraphs[p].text) > 50:
      #print((paragraphs[p].text))
      #print(f"TALKS ABOUT: {topic_detection(paragraphs[p].text)}")
      p_list.append(paragraphs[p].text)
      topic_list.append(topic_detection(paragraphs[p].text))
p_dictionary['Paragraphs'] = p_list
p_dictionary['Topics'] = topic_list

In [59]:
p_dictionary

{'Paragraphs': ['Home Depot beats earnings estimates but warns of "continued pressure" on consumers'],
 'Topics': [[]]}

In [60]:
medium_df = pd.DataFrame(p_dictionary)

In [ ]:
medium_df

In [18]:
medium_df['Character Count'] = medium_df['Paragraphs'].apply(lambda x: len(x))


# Start

In [6]:
model_id = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
label = pipeline('sentiment-analysis', model = model_id)

C:\Users\Cantek Student AM\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Cantek Student AM\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaFor

In [4]:
url = 'https://ca.finance.yahoo.com/crypto'
response = requests.get(url)
page_content = response.text
doc = BeautifulSoup(page_content, 'html.parser')

In [7]:
header = []
a_elements = doc.find_all('a', {'class': "js-content-viewer"})
for i in range (len(a_elements)):
    header.append(a_elements[i].text)
news_header_df = pd.DataFrame({'News_Header': header})

In [30]:
classification = []
for i in range (len(news_header_df.News_Header)):
    classification.append(label(news_header_df.News_Header[i]))

In [28]:
classification[4][0]['label']

'neutral'

In [32]:
labels = []
for i in range (len(classification)):
    labels.append(classification[i][0]['label'])

In [33]:
news_header_df['sentiment_analysis'] = labels

In [34]:
news_header_df

,News_Header,sentiment_analysis
0,MARKETS LIVE BLOGSTOCKS SLIDE AMID CHINA GLOOM...,positive
1,Bitcoin ETF ruling expected this week—and with...,positive
2,Bitcoin Is Languishing. What Could Shake Crypt...,negative
3,First Mover Americas: Europe's First Spot Bitc...,neutral
4,A tokenized version of Bitcoin is launching on...,neutral
5,"Bitcoin, Ether dip amid low volatility; Solana...",positive
6,The Secular Investment Case for Bitcoin and Cr...,neutral
7,Bitcoin price heads higher as US CPI data cont...,positive
8,UK watchdog raids 34 sites in crypto ATM crack...,neutral
9,Bitcoin rally falters as crypto pulls back yet...,negative
